# SQL MiniProject

Part 1: PHPMyAdmin interface through https://sql.springboard.com.  `Country Club` tables `Bookings`, `Facilities`, and `Members` used to answer questions below.  Code is provided that answers each question.

Part 2: `Country Club` tables are exported from PHPMyAdmin and connected to a local SQLite instance.  Questions below are answered using python code.

## Part 1

#### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

SELECT name FROM Facilities

WHERE COUNT(membercost = 0)

<img src="SQLminiprojectPics/Facilities-NoFee.PNG" alt="Facilities with no fee" title="Result" />

#### Q2: How many facilities do not charge a fee to members?

SELECT COUNT( name ) FROM Facilities

WHERE membercost =0

<img src="SQLminiprojectPics/Facilities-CountNoFee.PNG" alt="Number of facilities with no fee" title="Result" />

#### Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

SELECT facid, name, membercost, monthlymaintenance FROM Facilities

WHERE membercost <= monthlymaintenance * .2

<img src="SQLminiprojectPics/Facilities-FeeMaintenance.PNG" alt="Number of facilities with no fee" title="Result" />

#### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

SELECT * FROM Facilities

WHERE facid IN (1, 5)

<img src="SQLminiprojectPics/Facilities-Facid.PNG" alt="Number of facilities with no fee" title="Result" />

#### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

SELECT name, monthlymaintenance,

CASE WHEN monthlymaintenance >100

THEN 'expensive' ELSE 'cheap'

END AS pricerange FROM `Facilities`

<img src="SQLminiprojectPics/Facilities-PriceRange.PNG" alt="Number of facilities with no fee" title="Result" />

#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

SELECT firstname, surname FROM Members

<img src="SQLminiprojectPics/Members-Names.PNG" alt="Number of facilities with no fee" title="Result" />

#### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

SELECT sub.court, CONCAT( sub.first, ' ', sub.last ) AS name

FROM ( 

SELECT Facilities.name AS court, Members.firstname AS first, Members.surname AS last

FROM Bookings

INNER JOIN Facilities ON Bookings.facid = Facilities.facid

AND Facilities.name LIKE 'Tennis%'

INNER JOIN Members ON Bookings.memid = Members.memid) AS sub

GROUP BY sub.court, sub.first, sub.last

ORDER BY name

<img src="SQLminiprojectPics/Bookings-Members.PNG" alt="Number of facilities with no fee" title="Result" />

#### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

SELECT Facilities.name AS facility, CONCAT( Members.firstname, ' ', Members.surname ) AS name, 

CASE WHEN Bookings.memid =0

THEN Facilities.guestcost * Bookings.slots

ELSE Facilities.membercost * Bookings.slots

END AS cost

FROM Bookings

INNER JOIN Facilities ON Bookings.facid = Facilities.facid

AND Bookings.starttime LIKE '2012-09-14%'

AND (((Bookings.memid =0) AND (Facilities.guestcost * Bookings.slots >30))

OR ((Bookings.memid != 0) AND (Facilities.membercost * Bookings.slots >30)))

INNER JOIN Members ON Bookings.memid = Members.memid

ORDER BY cost DESC

<img src="SQLminiprojectPics/Bookings-DayCost.PNG" alt="Number of facilities with no fee" title="Result" />

#### This time, produce the same result as in Q8, but using a subquery.

SELECT * FROM (

SELECT Facilities.name AS facility, CONCAT( Members.firstname, ' ', Members.surname ) AS name,

CASE WHEN Bookings.memid =0

THEN Facilities.guestcost * Bookings.slots

ELSE Facilities.membercost * Bookings.slots

END AS cost

FROM Bookings

INNER JOIN Facilities ON Bookings.facid = Facilities.facid

AND Bookings.starttime LIKE '2012-09-14%'

INNER JOIN Members ON Bookings.memid = Members.memid) AS subquery

WHERE subquery.cost >30

ORDER BY subquery.cost DESC

<img src="SQLminiprojectPics/Bookings-Subquery.PNG" alt="Number of facilities with no fee" title="Result" />

## Part 2

#### Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook for the following questions. 

In [14]:
##### Method 1 for importing database (based on instruction to use sqlite3)

In [1]:
import sqlite3
from sqlite3 import Error

In [29]:
database = "sqlite_db_pythonsqlite.db"

try:
    conn = sqlite3.connect(database)
    print(sqlite3.version)
except Error as e:
    print(e)

2.6.0


In [30]:
cur = conn.cursor()
queryTest = """SELECT * FROM Facilities"""

# method to print out rows
cur.execute(queryTest)
rows = cur.fetchall()
for row in rows:
    print(row)
    
# create and print df using pandas
result = cur.execute(queryTest).fetchall()
pd.read_sql_query(queryTest, conn)

(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


##### Method 2 for importing database (based on tutorial from DataCamp recommended)

In [31]:
import pandas as pd
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///sqlite_db_pythonsqlite.db')


In [32]:
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [33]:
conn = engine.connect()
rs = conn.execute(queryTest)
df = pd.DataFrame(rs.fetchall())
df

,0,1,2,3,4,5
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [83]:
#close connection to df -- only run when ready to close!

conn.close()

#### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [39]:
queryQ10 = """SELECT name, totalrevenue
FROM(select facilities.name, 
SUM(CASE WHEN memid = 0 THEN slots * facilities.guestcost ELSE slots * membercost END) AS totalrevenue
FROM Bookings
INNER JOIN Facilities 
ON Bookings.facid = Facilities.facid
GROUP BY Facilities.name) AS selected_facilities WHERE totalrevenue <= 1000
ORDER BY totalrevenue"""

result = cur.execute(queryQ10).fetchall()
pd.read_sql_query(queryQ10, conn)

,name,totalrevenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


#### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [47]:
queryQ11 = """SELECT Members.firstname, Members.surname, Recommender.firstname, Recommender.surname FROM Members 
LEFT OUTER JOIN Members Recommender
ON Recommender.memid = Members.recommendedby
ORDER BY Members.surname, Members.firstname;
"""
result = cur.execute(queryQ11).fetchall()
pd.read_sql_query(queryQ11, conn)

,firstname,surname,firstname,surname
0,Florence,Bader,Ponder,Stibbons
1,Anne,Baker,Ponder,Stibbons
2,Timothy,Baker,Jemima,Farrell
3,Tim,Boothe,Tim,Rownam
4,Gerald,Butters,Darren,Smith
5,Joan,Coplin,Timothy,Baker
6,Erica,Crumpet,Tracy,Smith
7,Nancy,Dare,Janice,Joplette
8,David,Farrell,None,None
9,Jemima,Farrell,None,None


#### Q12: Find the facilities with their usage by member, but not guests

In [50]:
queryQ12 = """SELECT DISTINCT Facilities.name, Members.firstname, Members.surname FROM Members
INNER JOIN Bookings ON Members.memid = Bookings.memid
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
ORDER BY Facilities.name;
"""
result = cur.execute(queryQ12).fetchall()
pd.read_sql_query(queryQ12, conn)

,name,firstname,surname
0,Badminton Court,Darren,Smith
1,Badminton Court,GUEST,GUEST
2,Badminton Court,Gerald,Butters
3,Badminton Court,Tracy,Smith
4,Badminton Court,Nancy,Dare
...,...,...,...
206,Tennis Court 2,Jack,Smith
207,Tennis Court 2,Millicent,Purview
208,Tennis Court 2,Henrietta,Rumney
209,Tennis Court 2,John,Hunt


#### Q13: Find the facilities usage by month, but not guests 

In [82]:
#all data is from same year so no seperation by year needed.
#month selected by recommended code from stackoverflow: https://stackoverflow.com/questions/650480/get-month-from-datetime-in-sqlite
queryQ13 = """SELECT Facilities.name, strftime('%m',starttime) AS month, SUM(slots) AS times_booked FROM Bookings
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
GROUP BY month, Facilities.name;"""
result = cur.execute(queryQ13).fetchall()
pd.read_sql_query(queryQ13, conn)

,name,month,times_booked
0,Badminton Court,07,180
1,Massage Room 1,07,264
2,Massage Room 2,07,24
3,Pool Table,07,117
4,Snooker Table,07,156
5,Squash Court,07,164
6,Table Tennis,07,104
7,Tennis Court 1,07,270
8,Tennis Court 2,07,207
9,Badminton Court,08,459
